In [78]:
"""
Goal: Filter mobility data down to Saturday and Sunday 8am-8pm (times folks would go out to get groceries)
"""

import geopandas as gpd
import pandas as pd
import contextily as cx
import collections
import matplotlib.pyplot as plt
import datetime

#Can get rid of anything outside of 8am-8pm
#Then go down to weekends


In [117]:
#What it looks like to save one df
#G14Marchdf = pd.read_csv("/work/group/oit_research_data/mobility/data/data_DFW_2021/data/DFW_2021_03/G14_DFW_mobility_output.tsv", sep='\t')

In [80]:
#Parts of file path to use in for loops (iterating over different files)
start_file_name = "/work/group/oit_research_data/mobility/data/data_DFW_2021/data/DFW_2021_"
#need to add month in between start and mid
mid_file_name = "/"
#need to add row letter and row num in between mid and end
end_file_name = "_DFW_mobility_output.tsv"

In [128]:
#Filter individual file by Saturdays and Sunday 8am-8pm
def filter_weekends_by_time(df, start_hour, end_hour):
    df['DateTime'] = pd.to_datetime(df['DateTime'], utc =True)
    df = df[(df['DateTime'].dt.hour >= start_hour) & (df['DateTime'].dt.hour <= end_hour)]
    df['Date'] = pd.to_datetime(df['Date'], utc = True)
    df['Day Of Week'] = df['Date'].dt.day_name()

    weekend = ['Saturday', 'Sunday']
    
    df = df[(df['Day Of Week'].isin(weekend))]
    
    return df

In [129]:
#Returns a list of 12 dataframes, each containing a months worth of mobility data for the given cell (Row Letter and Row Number)
def loadFilteredDataByRowLetterAndNumber(row_letter, row_num):
    yearsMobilityData = []
    for i in range(1,13):
        df = pd.read_csv(start_file_name+"{:02d}".format(i)+ mid_file_name + row_letter + str(row_num)+ end_file_name, sep='\t')
        newDf = filter_weekends_by_time(df, 8, 20)
        gdf = gpd.GeoDataFrame(newDf, geometry=gpd.points_from_xy(newDf.Lon, newDf.Lat))
        yearsMobilityData.append(gdf)
    return yearsMobilityData

In [130]:
#Returns a list of lists, each with 12 months worth of mobility data for the cells listed in the dictionary given {row letter: row num, etc.}
def loadFilteredDataByRange(dictionary):
    listOfCellsData = []
    
    for item in dictionary:
        for thing in dictionary[item]:
            yearsMobileData = loadFilteredDataByRowLetterAndNumber(item, thing)
            listOfCellsData.append(yearsMobileData)

    return listOfCellsData

In [164]:
#Dictionaries to pull by 
dictionaryOfCells = {"H":[13,14], "G":[11,12,13,14],"F":[11,12,13,14], "E":[11,12,13]}
secondDictionaryOfCells = {"H":[11,12],"E":[14]}

In [ ]:
#Running the functions
#listOfFoodDesertCellsFilteredMobilityData = loadFilteredDataByRange(dictionaryOfCells)
#SecondListOfFoodDesertCellsFilteredMobilityData = loadFilteredDataByRange(secondDictionaryOfCells)

In [169]:
#Saving csv files of the filtered mobility data iteratively using list of cell names and list of lists of mobility data
def saveFilteredData(ListCells, ListData):
    i = 0
    for item in ListCells:
        for k in range (0,12):
                ListData[i][k].to_csv(item+"month"+str(k+1)+"filteredMobilityData.csv")
        i=i+1
        

In [ ]:
#Lists to save by
listOfCells = ['H13','H14','G11','G12','G13','G14','F11','F12','F13','F14','E11','E12','E13']
secondListOfCells = ['H11','H12','E14']

In [162]:
#Running the functions
#saveFilteredData(listOfCells, listOfFoodDesertCellsFilteredMobilityData)
#saveFilteredData(secondListOfCells, SecondListOfFoodDesertCellsFilteredMobilityData)